# Runnables 구조 검토

LCEL로 `runnable` 을 생성한 후에는 종종 이를 검사하여 어떤 일이 일어나고 있는지 더 잘 파악하고 싶을 것입니다.

이 노트북에서는 이를 수행하는 몇 가지 방법을 다룹니다.

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
# !pip install -qU faiss-cpu tiktoken

# 그래프를 그리기 위한 라이브러리 설치
# !pip install -qU grandalf

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    # 텍스트 데이터로부터 FAISS 벡터 저장소를 생성합니다.
    ["Teddy is an AI engineer who loves programming!"],
    embedding=OpenAIEmbeddings(),
)

# 벡터 저장소를 기반으로 retriever를 생성합니다.
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}  

Question: {question}"""

prompt = ChatPromptTemplate.from_template(
    template
)  # 템플릿을 기반으로 ChatPromptTemplate을 생성합니다.

model = ChatOpenAI(model="gpt-4o-mini")  # ChatOpenAI 모델을 초기화합니다.

# chain 을 생성합니다.
chain = (
    # 검색 컨텍스트와 질문을 지정합니다.
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt  # 프롬프트를 생성합니다.
    | model  # 언어 모델을 실행합니다.
    | StrOutputParser()  # 출력 결과를 문자열로 파싱합니다.
)

## 그래프 구성 확인

runnable의 그래프를 얻을 수 있습니다.


`chain.get_graph()` 메서드는 체인의 실행 그래프를 반환합니다.

- 이 메서드는 체인의 각 노드와 노드 간의 연결을 나타내는 그래프 객체를 반환합니다.
- 그래프의 노드는 체인의 각 단계를 나타내며, 에지(edge)는 단계 간의 데이터 흐름을 나타냅니다.


In [4]:
# 체인의 그래프에서 노드를 가져옵니다.
chain.get_graph().nodes

{'957892d1de0d4899b544ac1828462679': Node(id='957892d1de0d4899b544ac1828462679', name='Parallel<context,question>Input', data=<class 'langchain_core.runnables.base.RunnableParallel<context,question>Input'>, metadata=None),
 '3be8aeea9a934e5a8cb30a3e8ffb7229': Node(id='3be8aeea9a934e5a8cb30a3e8ffb7229', name='Parallel<context,question>Output', data=<class 'langchain_core.utils.pydantic.RunnableParallel<context,question>Output'>, metadata=None),
 '4762a53cea28426b9f62afc3e72d626c': Node(id='4762a53cea28426b9f62afc3e72d626c', name='VectorStoreRetriever', data=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f5b36191a90>, search_kwargs={}), metadata=None),
 'b04ce54b40574902a59d0db6422c0aa7': Node(id='b04ce54b40574902a59d0db6422c0aa7', name='Passthrough', data=RunnablePassthrough(), metadata=None),
 'f872ad7372f348c0bdfa81067f198746': Node(id='f872ad7372f348c0bdfa81067f198746', name='ChatPromptTemplate', data=Ch

In [5]:
# 체인의 그래프에서 엣지를 가져옵니다.
chain.get_graph().edges

[Edge(source='fdcbfd0844224a32a03eabef8633c193', target='60cbf01359ed419ba36c9a4396880193', data=None, conditional=False),
 Edge(source='60cbf01359ed419ba36c9a4396880193', target='7a2dea2ea9db437d838b5127ae57dd0f', data=None, conditional=False),
 Edge(source='fdcbfd0844224a32a03eabef8633c193', target='b263b3439467416d86defc99b707a28f', data=None, conditional=False),
 Edge(source='b263b3439467416d86defc99b707a28f', target='7a2dea2ea9db437d838b5127ae57dd0f', data=None, conditional=False),
 Edge(source='7a2dea2ea9db437d838b5127ae57dd0f', target='84e6cd43d6b546d98e834c82ad334722', data=None, conditional=False),
 Edge(source='84e6cd43d6b546d98e834c82ad334722', target='58baa974c0074f9da0f1bef077e9c08b', data=None, conditional=False),
 Edge(source='83adca92555e44f69c4f40cb6dfa7b14', target='9b5fde1e7519498882ba6a6b9921b775', data=None, conditional=False),
 Edge(source='58baa974c0074f9da0f1bef077e9c08b', target='83adca92555e44f69c4f40cb6dfa7b14', data=None, conditional=False)]

## 그래프 출력

그래프를 출력하면 이해하기 쉬운 형태로 표현할 수 있습니다.

비록 출력 결과가 매우 읽기 쉽지는 않지만, 출력을 통해 보다 이해하기 쉬운 형태로 그래프를 확인할 수 있습니다.


In [7]:
# 체인의 그래프를 ASCII 형식으로 출력합니다.
chain.get_graph().print_ascii()

           +---------------------------------+         
           | Parallel<context,question>Input |         
           +---------------------------------+         
                    **               **                
                 ***                   ***             
               **                         **           
+----------------------+              +-------------+  
| VectorStoreRetriever |              | Passthrough |  
+----------------------+              +-------------+  
                    **               **                
                      ***         ***                  
                         **     **                     
           +----------------------------------+        
           | Parallel<context,question>Output |        
           +----------------------------------+        
                             *                         
                             *                         
                             *                  

## 프롬프트 가져오기

체인에서 중요한 부분은 사용되는 프롬프트입니다.


`chain.get_prompts()` 메서드는 체인에서 사용되는 프롬프트(prompt) 객체의 리스트를 반환합니다.


In [8]:
chain.get_prompts()  # 체인에서 사용되는 프롬프트를 가져옵니다.

[ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based only on the following context:\n{context}  \n\nQuestion: {question}'), additional_kwargs={})])]